In [ ]:
#@title  { display-mode: "form" }
#@markdown #Authorise the google sheet
#@markdown Click the link and type the authorization code
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#@markdown #Functions

# imports 
import copy
import string
from openpyxl import load_workbook,  Workbook
import pandas as pd
from openpyxl import load_workbook,  Workbook
from oauth2client.crypt import MAX_TOKEN_LIFETIME_SECS

#@markdown def Google_sheet_to_list_dict(workbook, sheet_name): """get the data in a google sheet in a list of dictionaries"""
def Google_sheet_to_list_dict(workbook, sheet_name):
  """get the data in a google sheet in a list of dictionaries"""
  a = []
  sheet = wb_m.worksheet(sheet_name)
  data = sheet.get_all_values()
  heads = data[0]
  for i, item in enumerate(data):
    if i != 0:
      b = {}
      for j, jtem in enumerate(heads):
        b[jtem] = item[j]
      a.append(b)
  return a 

#@markdown def print1(a, b = "list of dict"): """print a list of dict"""
def print1(a, b = "list of dict"):
  """print a list of dict"""
  print(b)
  print("( len =",len(a), ")", end = " ")
  if len(a) != 0:
    print (a[0].keys())
  
  for i, item in enumerate(a):
    print(i, ":", item)
  print("")

#@markdown def print2(a, b, count = 1): """print a dict of list of dict"""
def print2(a, b , count = 1):
  """print a dict of list of dict"""
  print(b)
  for key, value in a.items():
    print("    ", key)
    for i in value:
      print(count, ":", i)
      count += 1 
  print("")

#@markdown def print3(a, b): """print a dict of a dict of list of dict"""
def print3(a, b):
  """print a dict of a dict of list of dict"""
  print(b)
  for key, value in a.items():
    print2(value, key)


#@markdown def print4(a, b): """ print a customezed think dont remember what"""
def print4(a, b):
  count = 1
  print(b)
  for i in a:
    print(to_excel(i["code"]), end=" : ")
    print(i["Order_id"])
    print(i["Order"])
    for j in i["Items"]:
      print(count, ":", j)
      count += 1
    print("")
  print("")  


#@markdown def Load_excel_file(file_path, sheet_name): """load excel file and return loaded sheet"""
def Load_excel_file(file_path, sheet_name):
  """load excel file and return loaded sheet"""
  wb = load_workbook(file_path)   
  sheet = wb[sheet_name]  
  return sheet

#@markdown def Make_lst_dict(sheet): """make a list of dictionaries, given a sheet"""
def Make_lst_dict(sheet):
  """make a list of dictionaries, given a sheet"""

  def Get_headings(sheet):
    """get the headings of a loaded sheet """
    a = []
    a.append("0")
    for i in range(1, sheet.max_column + 1):
      a.append(sheet.cell(row = 1, column = i).value)
    return a 
  headings = Get_headings(sheet)  

  a_list = []
  for i in range(2, sheet.max_row + 1):
    b_dict = {}
    for j in range (1, sheet.max_column + 1):
      b_dict[headings[j]] = sheet.cell(row = i, column = j).value
    a_list.append(b_dict)
  return a_list 

#@markdown def Make_qty_1(lst): """ duplicate the items Qty times"""
def Make_qty_1(lst):
  """ duplicate the items Qty times"""
  pannels_list_itter = copy.deepcopy(lst)
  pannels_list = copy.deepcopy(lst)

  count = 0 
  for i, item in enumerate(pannels_list_itter):
    count = count + 1
    if item["Qty"] < 2:
      pass
    else:
      no_of_copies = item["Qty"]
      #item["Qty"] = 1
      #pannels_list[i]["Qty"] = 1
      for j in range(no_of_copies - 1):
        pannels_list.insert(count, item.copy())
        count = count + 1
  return pannels_list

#@markdown def Get_order_details(cust_login, ord_login,  order): 
def Get_order_details(cust_login, ord_login,  order):
  """Get
  Order_id :  from variable order_id
  Customer_order_number : from ORDER_LOGIN
  Customer_order_name : from ORDER_LOGIN
  Description : from ORDER_LOGIN
  Type : from ORDER_LOGIN
  Order_confirm_date : from ORDER_LOGIN
  Customer_id : from ORDER_LOGIN
  Customer_name : from CUSTOMER_LOGIN
  """
  a = {}
  for i, item in enumerate(ord_login):
    if item["Order_id"] == str(order):
      a["Order_id"] = order
      a["Customer_order_no"] = item["Customer_order_no"]
      a["Customer_order_name"] = item["Customer_order_name"]
      a["Description"] = item["Description"]
      a["Type"] = item["Type"]
      a["Order_confirm_date"] = item["Order_confirm_date"]
      a["Customer_id"] = item["Customer_id"]
  for i , item in enumerate(cust_login):
      if item["Customer_id"] == str(a["Customer_id"]):
        a["Customer_name"] = item["Customer_name"]
  
  return a

#@markdown def get_last_plus1(pannel_sheet_list, col_name): """get the last column of the list of dict"""
def get_last_plus1(pannel_sheet_list, col_name):
  """get the last column of the list of dict"""
  a = []
  for i, item in enumerate(pannel_sheet_list):
    a.append(int(pannel_sheet_list[i][col_name]))
  return max(a) + 1

#@markdown def fill_data_other_than_packing(lst, pannel_start, ord_details): 
def fill_data_other_than_packing(lst, pannel_start, ord_details):
  pannels_list = copy.deepcopy(lst)

  order_qty_count = 1 
  for i , item in enumerate(pannels_list):
    item["Pannel_id"] = pannel_start
    pannel_start +=  1

    item["Order_id"] = ord_details["Order_id"]
    item["Customer_name"] = ord_details["Customer_name"]
    item["Customer_order_no"] = ord_details["Customer_order_no"]
    item["Customer_order_name"] = ord_details["Customer_order_name"]
    item["Customer_id"] = ord_details["Customer_id"]
    item["Qty"] = 1

    max_items = str(len(pannels_list))

    item["Order_qty"] = str(i+1) + "of" + max_items

    item["Size"] = str(item["Thickness"]) + "x" + str(item["Length"]) + "x" + str(item["Width"])
    a = ""
    if item["Drill"] == "Y":
      a += "D"
    if item["Mdrill"] == "Y":
      a += "M"
    if item["Drill"] != "Y" and item["Mdrill"] != "Y":
      a += "NA"
    item["Boring_or_not"] = a
  return pannels_list

#@markdown def Divide_Order_sub_part(lst): """divide accourinig to Order_sub_part"""
def Divide_Order_sub_part(lst):
  """divide accourinig to Order_sub_part"""
  pannels_list = copy.deepcopy(lst)
  a = {}
  for i, item in enumerate(pannels_list):
    if item["Order_sub_part"] in a:
      a[item["Order_sub_part"]].append(item)
    else: 
      a[item["Order_sub_part"]] = list()
      a[item["Order_sub_part"]].append(item)
  return copy.deepcopy(a)

#@markdown def Divide_and_merge_packing_types(lst, type_2_set): """divide into groups, firmat and remearge and sort"""
def Divide_and_merge_packing_types(lst, type_2_set):
  """divide into groups, firmat and remearge and sort"""
  pannels_list = copy.deepcopy(lst)
  a = {}

  for x, xtem in pannels_list.items():
    type_1 = list()
    type_2 = list()
    
    # divide into two types 
    for i, item in enumerate(xtem):
      if item["Thickness"] < 16:
        type_2.append(item)
      elif item["Code"][:2] in type_2_set:
        type_2.append(item)
      else:
        type_1.append(item)

    # format type 2 
    type_2_dict = {}
    for i, item in enumerate(type_2):
      if item["Code"][:2] in type_2_dict:
        type_2_dict[item["Code"][:2]].append(item)
      else: 
        type_2_dict[item["Code"][:2]] = list()
        type_2_dict[item["Code"][:2]].append(item)

    # format type 1
    type_1_dict = {}
    for i, item in enumerate(type_1):
      if item["Code"] in type_1_dict:
        type_1_dict[item["Code"]].append(item)
      else: 
        type_1_dict[item["Code"]] = list()
        type_1_dict[item["Code"]].append(item)

    #merge both types 
    for key, value in type_2_dict.items():
      type_1_dict[key] = value
      
    a[x] = type_1_dict
  return copy.deepcopy(a)

#@markdown def sort_dict_dict_list_dict(dic1): """sort a list inside a dictionary inside a dictionary based on the dict value inside the list items"""
def sort_dict_dict_list_dict(dic1):
  """sort a list inside a dictionary inside a dictionary based on the dict value inside the list items"""
  to_sort = copy.deepcopy(dic1)
  for xkeys, xvalues in to_sort.items():

    for keys, value in xvalues.items():
      ab = sorted(xvalues[keys], key=lambda d: d['Length'], reverse=True)
      xvalues[keys] = copy.deepcopy(ab)
    to_sort[xkeys] = copy.deepcopy(xvalues)
  return copy.deepcopy(to_sort)

#@markdown def merge_last_2_dict(dic): """merge the last two dicts with '-' so that the code doesnt have to be changed """
def merge_last_2_dict(dic):
  """merge the last two dicts with '-' so that the code doesnt have to be changed """
  a = copy.deepcopy(dic)
  b = {}
  for key, value in a.items():
    for ikey, ivalue in value.items():
      b[key + "#" + ikey ] = ivalue
  return b


#@markdown def to_excel(num): """convert nbumber to excel style code"""
def to_excel(num):
  """convert nbumber to excel style code"""
  def divmod_excel(n):
    a, b = divmod(n, 26)
    if b == 0:
      return a - 1, b + 26
    return a, b

  chars = []
  while num > 0:
      num, d = divmod_excel(num)
      chars.append(string.ascii_uppercase[d - 1])
  return ''.join(reversed(chars))


#@markdown def Count_items_packed(a):
def Count_items_packed(a):
  count = 0 
  for item in a:
    if item["Packing_qty"] == "Packed":
      count += 1 
  return count

#@markdown def Count_incomplete_items(a):
def Count_incomplete_items(a):
  count = 0 
  for item in a:
    if item["Packing_qty"] != "Packed":
      count += 1
  return count 

#@markdown def listToString(s): 
def listToString(s): 
    str1 = ","  
    return (str1.join(s))

# 5 layers or weight 20
#@markdown def Make_packing(lst):
def Make_packing(lst):
  pannels_sorted = copy.deepcopy(lst)
  packing_list = []

  stack = {}
  stack["code"] = len(packing_list) + 1
  stack["Items"] = list()
  stack["Space"] = list()
  stack["Max"] = 720
  stack["Order"] = list()
  stack["Order_id"] = list()
  stack["Weight"] = float(0)

  for a in pannels_sorted.values():
    while True:
      if Count_incomplete_items(a) < 1:
        # save stack 
        packing_list.append(copy.deepcopy(stack))
        # create new stack
        stack = {}
        stack["code"] = len(packing_list) + 1
        stack["Items"] = list()
        stack["Space"] = list()
        stack["Max"] = 720
        stack["Order"] = list()
        stack["Order_id"] = list()
        stack["Weight"] = float(0)
        break 

      if len(stack["Space"]) > 5 or stack["Weight"] > 20:
        # save stack 
        packing_list.append(copy.deepcopy(stack))
        # create new stack
        stack = {}
        stack["code"] = len(packing_list) + 1
        stack["Items"] = list()
        stack["Space"] = list()
        stack["Max"] = 720
        stack["Order"] = list()
        stack["Order_id"] = list()
        stack["Weight"] = float(0)
        continue 

      else:  
        # loop over items 
        for item in a:
          if item["Packing_qty"] == "Packed":
            continue
          else:
            if len(stack["Items"]) < 1:
              # make max length, and space list
              if item["Length"] > 720:
                stack["Max"] = item["Length"]
                stack["Space"].append(0)
              else: 
                stack["Max"] = 720
                stack["Space"].append(720 - item["Length"])

              # add item to stack /  change to packed / make order
              stack["Items"].append(item)
              item["Packing_qty"] = "Packed"
              stack["Order"].append([item["Length"]])
              stack["Order_id"].append([item["Pannel_id"]])
              stack["Weight"] = item["Length"] * item["Width"] * item["Thickness"] * 600 /1000/1000/1000
              continue 
            else:
              # create check list 
              stack["Check"] = []

              # create check list 
              for Spaces in stack["Space"]:
                if Spaces < item["Length"]:
                  stack["Check"].append(9999999999)
                else: 
                  stack["Check"].append(Spaces - item["Length"])
              min_value = min(stack["Check"])
              min_index = stack["Check"].index(min_value)

              if min_value == 9999999999:
                # make Space / make order
                stack["Space"].append(stack["Max"] - item["Length"])
                stack["Order"].append([item["Length"]])
                stack["Order_id"].append([item["Pannel_id"]])

              else: 
                # make Space . prder
                stack["Space"][min_index] -= item["Length"]
                stack["Order"][min_index].append(item["Length"])
                stack["Order_id"][min_index].append(item["Pannel_id"])

              # add item to stack / packed
              stack["Weight"] += item["Length"] * item["Width"] * item["Thickness"] * 600 /1000/1000/1000
              stack["Items"].append(item)
              item["Packing_qty"] = "Packed"

              if len(stack["Space"]) > 5:
                # save stack 
                packing_list.append(copy.deepcopy(stack))
                # create new stack
                stack = {}
                stack["code"] = len(packing_list) + 1
                stack["Items"] = list()
                stack["Space"] = list()
                stack["Max"] = 720
                stack["Order"] = list()
                stack["Order_id"] = list()
                stack["Weight"] = float(0)
                break

              else: 
                continue 

  return packing_list

#@markdown def Correct_packing(lst, packing_start): """correct the mistakes in packing"""
def Correct_packing(lst, packing_start):
  """correct the mistakes in packing"""
  a = []
  for i, item in enumerate(lst):
    if len(item["Items"]) != 0:
      item["code"] = i + 1
      a.append(item)

  count = packing_start
  ccount = 1
  for i, item in enumerate(a):
    item["code"] = i+1
    for j , jtem in enumerate(item["Items"]):
      jtem["Pannel_qty"] = to_excel(ccount) + " " +  str(j + 1) + "of" + str(len(item["Items"])) 
      jtem["Packing_id"] = "p" + str(count) 
      jtem["Packing_qty"] = str(ccount) + "of" + str(len(a))
      jtem["Packing_print"] = jtem["Pannel_qty"] +" "+ jtem["Packing_qty"]
    count += 1 
    ccount += 1

  return copy.deepcopy(a)


#@markdown def get_last_plus1_p(packet_sheet_list, col_name): """get the last column of the list of dict"""
def get_last_plus1_p(packet_sheet_list, col_name):
  """get the last column of the list of dict"""
  a = []
  for i, item in enumerate(packet_sheet_list):
    a.append(int(packet_sheet_list[i][col_name][1:]))
  return max(a) + 1

#@markdown def Get_mats_pannels(lst):
def Get_mats_pannels(lst):
  a = []
  for i, item in enumerate(lst):
    for j, jtem in enumerate(item["Items"]):
      a.append(jtem)

  return copy.deepcopy(a)


#@markdown def list_of_list_to_string(lst):
def list_of_list_to_string(lst):
  a = "["
  for i in lst:
    b = "[" 
    for j in i:
      b += str(j) 
      b += "," 
    b += "]"
    a += b
  a += "]"
  return a 

#@markdown def Get_matts_packing(lst, order_detais):
def Get_matts_packing(lst, order_detais):
  a = []
  for i, item in enumerate(lst):
    b = {}
    b["Packing_id"] = item["Items"][0]["Packing_id"]
    b["Order_id"] = order_detais["Order_id"]
    b["Customer_id"] = order_detais["Customer_id"]
    b["Customer_name"] = order_detais["Customer_name"]

    b["Customer_order_no"] = order_detais["Customer_order_no"]
    b["Customer_order_name"] = order_detais["Customer_order_name"]

    b["Description"] = order_detais["Description"]
    b["Type"] = order_detais["Type"]
    b["Order_confirm_date"] = order_detais["Order_confirm_date"]

    b["Order_sub_part"] = item["Items"][0]["Order_sub_part"]
    b["Product_name"] = item["Items"][0]["Product_name"] 

    b["Pannels_list"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Pannels_list"].append(str(jtem["Pannel_id"]))
    b["Pannels_list"] = listToString(b["Pannels_list"])

    b["Packing_order"] = list_of_list_to_string(item["Order_id"])

    b["Part_name"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Part_name"].append(str(jtem["Part_name"]))
    b["Part_name"] = listToString(b["Part_name"])

    b["Size"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Size"].append(str(jtem["Size"]))
    b["Size"] = listToString(b["Size"])

    b["Colour"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Colour"].append(str(jtem["Colour"]))
    b["Colour"] = listToString(b["Colour"])

    b["Code"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Code"].append(str(jtem["Code"]))
    b["Code"] = listToString(b["Code"])

    b["Packing_print"] = list()
    for j, jtem in enumerate(item["Items"]):
      b["Packing_print"].append(str(jtem["Packing_print"]))
    b["Packing_print"] = listToString(b["Packing_print"])

    a.append(b)
  return a 


#@markdown def write(file_path, sheet_name, dictionary):
def write(file_path, sheet_name, dictionary):
  wb = load_workbook(file_path)
  sheet = wb[sheet_name]

  headers = [x for x in dictionary[0]]

  for index, value in enumerate(headers):
      sheet.cell(row=1, column=index+1).value = value

  for i, x in enumerate(dictionary):
      for idx,value in enumerate(x.values()):
          sheet.cell(row=i+2, column=idx+1).value = value

  wb.save(file_path)


#@markdown def create_null(dic): """create null items for google sheets"""
def create_null(dic):
  """create null items"""
  a = copy.deepcopy(dic)
  b = []
  for i, item in enumerate(a):
    c = []
    for key, value in item.items():
      c.append("")
    b.append(c)
  return b 


#@markdown def numberToLetters(q): """Helper function to convert number of column to its index, like 10 -> 'A'"""
def numberToLetters(q):
    """
    Helper function to convert number of column to its index, like 10 -> 'A'
    """
    q = q - 1
    result = ''
    while q >= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

#@markdown def colrow_to_A1(col, row):
def colrow_to_A1(col, row):
    return numberToLetters(col)+str(row)

#@markdown def update_sheet(ws, rows, left=1, top=1): updates the google spreadsheet with given table
def update_sheet(ws, rows, left=1, top=1):
    """
    updates the google spreadsheet with given table
    - ws is gspread.models.Worksheet object
    - rows is a table (list of lists)
    - left is the number of the first column in the target document (beginning with 1)
    - top is the number of first row in the target document (beginning with 1)
    """
    if len(rows) != 0:

      # number of rows and columns
      num_lines, num_columns = len(rows), len(rows[0])

      # selection of the range that will be updated
      cell_list = ws.range(
          colrow_to_A1(left,top)+':'+colrow_to_A1(left+num_columns-1, top+num_lines-1)
      )

      # modifying the values in the range

      for cell in cell_list:
          val = rows[cell.row-top][cell.col-left]
          cell.value = val

      # update in batch
      ws.update_cells(cell_list)

#@markdown def ldll(dic): """convert a list of dict to a list of list"""
def ldll(dic):
  """convert a list of dict to a list of list"""
  a = copy.deepcopy(dic)
  b = []
  for i, item in enumerate(a):
    c = []
    for key, value in item.items():
      c.append(value)
    b.append(c)
  return b 

#@markdown def Calculate_the_data(excel_file_name, sheet_name_packing, sheet_name_pannels, order_id, packing_combined): [B1, B2]
def Calculate_the_data(excel_file_name, sheet_name_packing, 
                       sheet_name_pannels, order_id, packing_combined):
  A1 = Make_lst_dict(Load_excel_file(excel_file_name, sheet_name_pannels)) # print1
  order_detais = Get_order_details(customer_login, order_login,  order_id) 

  A2 = Make_qty_1(A1) # print1
  pannel_id_start = get_last_plus1(pannels, "Pannel_id")

  A3 = fill_data_other_than_packing(A2, pannel_id_start, order_detais) # print1
  A4 = Divide_Order_sub_part(A3) # print2
  A5 = Divide_and_merge_packing_types(A4, packing_combined) # print3
  A6 = sort_dict_dict_list_dict(A5) # print3
  A7 = merge_last_2_dict(A6) # print2
  A8 = Make_packing(A7) # print1

  packing_id_start = get_last_plus1_p(packets, "Packet_id")
  A9  = Correct_packing(A8, packing_id_start) # print4

  B1 = Get_mats_pannels(A9) # print1
  B2 = Get_matts_packing(A9, order_detais) # print1

  write(excel_file_name, sheet_name_pannels, B1)
  write(excel_file_name, sheet_name_packing, B2)
  return [B1, B2]


In [ ]:
#@markdown #Test google sheet 

link_for_google_sheet_main = 'https://docs.google.com/spreadsheets/d/18pI7ApUVvbI-K9zAAqOubHW-eFumNtnZFmchexJyX_U/edit#gid=1610684638' #@param {type:"string"}
wb_m = gc.open_by_url(link_for_google_sheet_main)
#@markdown customer_login, order_login, status_order, pannels, packets, instructions
# load the google sheets main
customer_login = Google_sheet_to_list_dict(wb_m, "CUSTOMER_LOGIN")
order_login = Google_sheet_to_list_dict(wb_m, "ORDER_LOGIN")
status_order = Google_sheet_to_list_dict(wb_m, "STATUS_ORDER")
pannels = Google_sheet_to_list_dict(wb_m, "PANNELS")
packets = Google_sheet_to_list_dict(wb_m, "PACKETS")
instructions = Google_sheet_to_list_dict(wb_m, "INSTRUCTIONS")

#@markdown beansaw, beansaw_t, edgebend_t, edgebend_t, drill, drill_t, mdrill, mdrill_t, qc, qc_t, packing, packing_t
# load the google sheet track
beansaw = Google_sheet_to_list_dict(wb_m, "BEANSAW")
beansaw_t = Google_sheet_to_list_dict(wb_m, "BEANSAW_T")
edgebend = Google_sheet_to_list_dict(wb_m, "EDGEBEND")
edgebend_t = Google_sheet_to_list_dict(wb_m, "EDGEBEND_T")
drill = Google_sheet_to_list_dict(wb_m, "DRILL")
drill_t = Google_sheet_to_list_dict(wb_m, "DRILL_T")
mdrill = Google_sheet_to_list_dict(wb_m, "MDRILL")
mdrill_t = Google_sheet_to_list_dict(wb_m, "MDRILL_T")
qc = Google_sheet_to_list_dict(wb_m, "QC")
qc_t = Google_sheet_to_list_dict(wb_m, "QC_T")
packing = Google_sheet_to_list_dict(wb_m, "PACKING")
packing_t = Google_sheet_to_list_dict(wb_m, "PACKING_T")

In [ ]:
#@title ENTER THE VALUES

excel_file_name = "Makeup Firfly.xlsx" #@param {type:"string"}
sheet_name_pannels = "MATS_PANNELS" #@param {type:"string"}
sheet_name_packing = "MATS_PACKING" #@param {type:"string"}

order_id =  2 #@param {type:"integer"}

#@markdown Pannel codes that need to be combined into one 
#maximum_paking_height_in_18mm_pannels = 5 #@param {type:"integer"}
packing_combined = 'BK,SH,CP' #@param {type:"string"}
packing_combined = set(packing_combined.split(","))

In [ ]:
#@markdown #Calculate the data 
Z1 = Calculate_the_data(excel_file_name, sheet_name_packing, 
                        sheet_name_pannels, order_id, packing_combined)
Z1 = Calculate_the_data(excel_file_name, sheet_name_packing, 
                        sheet_name_pannels, order_id, packing_combined)

print1(Z1[0])
print1(Z1[1])

B1 = Z1[0]
B2 = Z1[1]

list of dict
( len = 13 ) dict_keys(['Pannel_id', 'Order_id', 'Customer_id', 'Customer_name', 'Customer_order_no', 'Customer_order_name', 'Size', 'Packing_print', 'Boring_or_not', 'Order_sub_part', 'Product_name', 'Type', 'Thickness', 'Length', 'Width', 'Qty', 'Align', 'Part_name', 'Colour', 'Code', 'Drill', 'Mdrill', 'Remark_Text', 'Instruction_1', 'Instruction_2', 'Eb_colour', 'Eb_thickness', 'Eb_left', 'Eb_right', 'Eb_top', 'Eb_bottom', 'Packing_id', 'Pannel_qty', 'Packing_qty', 'Order_qty', 'Drilling_file_path_front', 'Drilling_file_path_back'])
0 : {'Pannel_id': 1, 'Order_id': 2, 'Customer_id': '5', 'Customer_name': 'something', 'Customer_order_no': '58', 'Customer_order_name': 'rohit', 'Size': '18x742x50', 'Packing_print': 'A 1of6 1of3', 'Boring_or_not': 'NA', 'Order_sub_part': 'bedroom', 'Product_name': 'firefox', 'Type': 'vanity', 'Thickness': 18, 'Length': 742, 'Width': 50, 'Qty': 1, 'Align': None, 'Part_name': 'Drawer Bottom Patti', 'Colour': 'Frosty White PB', 'Code': 'MF01'

In [ ]:
#@markdown #Load the file again 
Z1 = Calculate_the_data(excel_file_name, sheet_name_packing, 
                        sheet_name_pannels, order_id, packing_combined)
Z1 = Calculate_the_data(excel_file_name, sheet_name_packing, 
                        sheet_name_pannels, order_id, packing_combined)
print1(Z1[0])
print1(Z1[1])

B1 = Z1[0]
B2 = Z1[1]

list of dict
( len = 13 ) dict_keys(['Pannel_id', 'Order_id', 'Customer_id', 'Customer_name', 'Customer_order_no', 'Customer_order_name', 'Size', 'Packing_print', 'Boring_or_not', 'Order_sub_part', 'Product_name', 'Type', 'Thickness', 'Length', 'Width', 'Qty', 'Align', 'Part_name', 'Colour', 'Code', 'Drill', 'Mdrill', 'Remark_Text', 'Instruction_1', 'Instruction_2', 'Eb_colour', 'Eb_thickness', 'Eb_left', 'Eb_right', 'Eb_top', 'Eb_bottom', 'Packing_id', 'Pannel_qty', 'Packing_qty', 'Order_qty', 'Drilling_file_path_front', 'Drilling_file_path_back'])
0 : {'Pannel_id': 1, 'Order_id': 2, 'Customer_id': '5', 'Customer_name': 'something', 'Customer_order_no': '58', 'Customer_order_name': 'rohit', 'Size': '18x742x50', 'Packing_print': 'A 1of6 1of3', 'Boring_or_not': 'NA', 'Order_sub_part': 'bedroom', 'Product_name': 'firefox', 'Type': 'vanity', 'Thickness': 18, 'Length': 742, 'Width': 50, 'Qty': 1, 'Align': None, 'Part_name': 'Drawer Bottom Patti', 'Colour': 'Frosty White PB', 'Code': 'MF01'

In [ ]:
#@markdown #Copy to google sheets 
sheet1  = wb_m.worksheet("PANNELS")
sheet2  = wb_m.worksheet("PACKETS")
sheet3  = wb_m.worksheet("BEANSAW")
sheet4  = wb_m.worksheet("BEANSAW_T")
sheet5  = wb_m.worksheet("EDGEBEND")
sheet6  = wb_m.worksheet("EDGEBEND_T")
sheet7  = wb_m.worksheet("DRILL")
sheet8  = wb_m.worksheet("DRILL_T")
sheet9  = wb_m.worksheet("MDRILL")
sheet10 = wb_m.worksheet("MDRILL_T")
sheet11 = wb_m.worksheet("QC")
sheet12 = wb_m.worksheet("QC_T")
sheet13 = wb_m.worksheet("PACKING")
sheet14 = wb_m.worksheet("PACKING_T")

B4  = list() #saw
B6  = list() #edge
B8  = list() #drill
B10 = list() #mdrill
B12 = list() #qc
B14 = list() #pack
for item in B1:
  a = {}
  a["Pannel_id"] = item["Pannel_id"]
  B4.append(a)
  B12.append(a)
  if item["Eb_thickness"] != None:
    B6.append(a)
  if item["Drill"] != None:
    B8.append(a)
  if item["Mdrill"] != None:
    B10.append(a)

for item in B2:
  a = {}
  a["Packing_id"] = item["Packing_id"]
  B14.append(a)
  

update_sheet(sheet1, ldll(B1), left=1, top=len(pannels) + 2)
update_sheet(sheet2, ldll(B2), left=1, top=len(packets) + 2)

update_sheet(sheet4, ldll(B4), left=1, top=len(beansaw_t) + 2)
update_sheet(sheet6, ldll(B6), left=1, top=len(edgebend_t) + 2)
update_sheet(sheet8, ldll(B8), left=1, top=len(drill_t) + 2)
update_sheet(sheet10, ldll(B10), left=1, top=len(mdrill_t) + 2)
update_sheet(sheet12, ldll(B12), left=1, top=len(qc_t) + 2)
update_sheet(sheet14, ldll(B14), left=1, top=len(packing_t) + 2)

In [ ]:
#@markdown #remove the values from the sheet if something went wrong

update_sheet(sheet1, create_null(B1), left=1, top=len(pannels) + 2)
update_sheet(sheet2, create_null(B2), left=1, top=len(packets) + 2)

update_sheet(sheet4, create_null(B4), left=1, top=len(beansaw_t) + 2)
update_sheet(sheet6, create_null(B6), left=1, top=len(edgebend_t) + 2)
update_sheet(sheet8, create_null(B8), left=1, top=len(drill_t) + 2)
update_sheet(sheet10, create_null(B10), left=1, top=len(mdrill_t) + 2)
update_sheet(sheet12, create_null(B12), left=1, top=len(qc_t) + 2)
update_sheet(sheet14, create_null(B14), left=1, top=len(packing_t) + 2)